In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import scrapy
#引入库

In [2]:
headers = {
'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0 '          
}
#获取请求头信息

In [3]:
class RentSpider(scrapy.Spider):
    name = 'rent_spider'
    start_urls = ['https://km.58.com/chuzu/?PGTID=0d3090a7-0021-d968-f927-b8ab41a5fd6e&ClickID=4',
                  'https://yx.58.com/chuzu/',
                  'https://dali.58.com/chuzu/',
                  'https://bs.58.com/chuzu/',
                  'https://qj.58.com/chuzu/']

    def parse(self, response):
        title_tags = response.css('a.strongbox')
        money_tags = response.css('div.money')
        room_tags = response.css('p.room')

        data = []
        for title_tag, money_tag, room_tag in zip(title_tags, money_tags, room_tags):
            title = title_tag.css('::text').get().strip() if title_tag else "N/A"
            money = money_tag.css('::text').get().strip() if money_tag else 'N/A'
            room = room_tag.css('::text').get().strip() if room_tag else "N/A"
            data.append([money, title, room])

        df = pd.DataFrame(data, columns=['价格', '租房信息', '房型'])
        yield {'data': df}

    def closed(self, reason):
        cities = ['昆明', '玉溪', '大理', '保山', '曲靖']

        for i, city in enumerate(cities):
            file_name = f'{city}.csv'
            data = self.crawler.stats.get_value('data', None)

            if data:
                data[i].to_csv(file_name, index=False)
                self.logger.info(f'Saved data to {file_name}')
            else:
                self.logger.error('No data found to save')

In [4]:
# 定义 URL 列表
urls = ['https://km.58.com/chuzu/',
        'https://yx.58.com/chuzu/',
        'https://dali.58.com/chuzu/',
        'https://bs.58.com/chuzu/',
        'https://qj.58.com/chuzu/']

# 假设headers已经定义好了
for index, url in enumerate(urls):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_tags = soup.find_all('a', class_='strongbox')
    money_tags = soup.find_all('div', class_='money')
    room_tags = soup.find_all('p', class_='room')

    if response.status_code == 200:
        print("请求成功")

        data = []
        for title_tag, money_tag, room_tag in zip(title_tags, money_tags, room_tags):
            title = title_tag.text.strip() if title_tag else "N/A"
            money = money_tag.text.strip() if money_tag else 'N/A'
            room = room_tag.text.strip() if room_tag else "N/A"
            data.append([money, title, room])

        df = pd.DataFrame(data, columns=['价格', '租房信息', '房型'])
        print(df)

        # 根据索引保存为对应城市的CSV文件
        city_names = ['昆明', '玉溪', '大理', '保山', '曲靖']
        city_name = city_names[index]
        filename = f"{city_name}.csv"
        df.to_csv(filename, index=False)

        print(f"数据已保存到 '{filename}' 文件")
    else:
        print("请求失败，状态码：", response.status_code)

请求成功
          价格                             租房信息  \
0     900元/月  合租 | 直租月付 万彩城新退主卧独卫 龙泉路 财大附近 同德   
1     500元/月  合租 | 毕业季特价房 无中介月付 近爱琴海购物中心 公园19   
2    1900元/月  整租 | 日新地铁口旁 小区中间位置！！带家具家电 有电梯 航   
3    1650元/月  整租 | 晓东村地铁站旁 紫金中心 回来世界 世纪金缘 拎包入   
4    2300元/月  整租 | 吾悦广场旁精装三室带全套家具家电南北通透近地铁近商圈   
..       ...                              ...   
120  1250元/月  整租 | 吴井路一环边百富琪旁广电小区3楼中装二室出租 带部份   
121  1900元/月             整租 | 世博园大门口东方玫瑰园3房出租   
122  1800元/月     整租 | 北市区 霖雨桥地铁口 江东小康城3房 房东急租   
123  1700元/月  整租 | 东华延安医院香博苑精装一室loft公寓包宽带民用水电   
124  1600元/月   整租 | 近海乐世界 万科公园城市精装端头三房出租  电梯房   

                                    房型  
0    主卧(2室)                        30㎡  
1    次卧(3室)                        26㎡  
2     3室                        85.41㎡  
3        1室                        50㎡  
4    3室                        102.23㎡  
..                                 ...  
120      2室                        65㎡  
121  3室                        123.28㎡  
122  3室                        118.73

In [5]:
city_names = ['昆明', '玉溪', '大理', '保山', '曲靖']
average_prices = []
total_zhengzu = []
total_hezu = []
top_five_list = []

for city_name in city_names:
    filename = f"{city_name}.csv"
    df = pd.read_csv(filename)

    # 计算平均房价
    average_price = df['价格'].str.extract(r'(\d+)', expand=False).astype(float).mean()
    average_prices.append(average_price)

    # 获取整租和合租的数量
    total_zhengzu.append(df[df['租房信息'].str.contains('整租')].shape[0])
    total_hezu.append(df[df['租房信息'].str.contains('合租')].shape[0])

    # 找出房价排名前五的行信息
    top_five = df.sort_values(by='价格', ascending=True).head(5)
    top_five_list.append(top_five)

# 打印每个城市的平均房价
for city, avg_price in zip(city_names, average_prices):
    print(f"{city}的平均房价为：{avg_price:.2f} 元")

# 打印每个城市的整租和合租数量
for city, zhengzu, hezu in zip(city_names, total_zhengzu, total_hezu):
    print(f"{city}的整租数量为：{zhengzu}，合租数量为：{hezu}")

# 打印每个城市房价排名前五的行信息
for city, top_five in zip(city_names, top_five_list):
    print(f"{city}房价排名前五的行信息：")
    print(top_five)
    print("\n")

昆明的平均房价为：1534.34 元
玉溪的平均房价为：nan 元
大理的平均房价为：nan 元
保山的平均房价为：nan 元
曲靖的平均房价为：nan 元
昆明的整租数量为：96，合租数量为：29
玉溪的整租数量为：0，合租数量为：0
大理的整租数量为：0，合租数量为：0
保山的整租数量为：0，合租数量为：0
曲靖的整租数量为：0，合租数量为：0
昆明房价排名前五的行信息：
         价格                             租房信息                                房型
43  1000元/月  合租 | 广福城雅福园 奥宸财富中心 润城 恒宇商业商场 南悦   (3室)                        30㎡
30  1050元/月  整租 | 西城时代十四中学旁团山欣城多套俩室带空房出租民水民电  2室                        65.48㎡
46  1050元/月  整租 | 五华区西城时代旁团山欣城多套俩室出租民水民电通天然气   2室                        65.2㎡
79  1050元/月  整租 | 西城时代旁团山欣城多套俩室带空房带家具家电出租民水民  2室                        64.15㎡
36  1100元/月                整租 | 斗南地铁口包物业押一付一  1室                        62.40㎡


玉溪房价排名前五的行信息：
Empty DataFrame
Columns: [价格, 租房信息, 房型]
Index: []


大理房价排名前五的行信息：
Empty DataFrame
Columns: [价格, 租房信息, 房型]
Index: []


保山房价排名前五的行信息：
Empty DataFrame
Columns: [价格, 租房信息, 房型]
Index: []


曲靖房价排名前五的行信息：
Empty DataFrame
Columns: [价格, 租房信息, 房型]
Index: []


